### Design and Analysis of Algorithms Semester Project
22I-0744    Aadil Umair CS-C
22I-0908    Salman Umar CS-C
22I-1095    Danish Atif CS-C

README BEFORE RUNNING
- This code takes several days to run on PC due to author's experimentation rules
- Therfore, the rules have been updated so that the code completes running in 2-3 hours
- The results of Algo 6 and Algo 7 are displayed in the last 2 cells
- They show the PS, CS and SR values of each funtion tested with an epsilon value with Algo 6 and Algo 7

WHAT THE RESULTS MEAN
- PR means peak ratio and is defined as the ratio of all global optima found *by my algorithm* to the *amount that should have been found* 
- SR means the ratio of *successful runs* to *all runs* for each function
- CS means the average number of fitness evaluations per run per function to find all the global optima

EXPERIMENTAL SETTINGS AND MY DEVIATIONS
- Algo 6 and Algo 7 are encapuslated in several loops
- The outermost loop iterates through each function (Normal: *All 20*   My Code: *First 10*)
- The next loop iterates through each epsilon in the epsilon set (Normal: *All 5*      My code: *1e-04*)
- The next loop runs the function a set number of runs just for numerical accuracy (Normal: *51 runs*     My Code: *1 run*)
- The next loop allows the function to run until all global optima are found OR if all global optima are not found then for a maximum number of fitness evaluations as allowed by the CEC2013 library for that function
- Within this while loop, Algo 6 or 7 is called to generate a solution set (called population in the code below), the fitness evaluations of that set and number of fitness evaluations it took to execute
- Then the CEC2013 library checks the amount of global optima present in the sol set and compares it to the amount that should be present
- The remaining PR, SR and CS calculations are trivial to calculate and can be understood by looking at the code below

In [1]:
from cec2013.cec2013 import *
import numpy as np
import math

In [2]:
########## MISC HELPER FUNCTIONS

#Euclidian Distance
def Euclidian_distance(vector1, vector2, dim):
    sum_squared_diff = sum((vector1[i] - vector2[i])**2 for i in range(dim))
    return math.sqrt(sum_squared_diff)

#get FSmax and FSmin of a niche
def FSmaxMinOfNiche(niche, f):
    FSiList = []
    for point in niche:
        if point.size==1:
            FSiList = np.append(FSiList, f.evaluate(np.array([point])))
        else:
            FSiList = np.append(FSiList, f.evaluate(np.array(point)))
    FSiMax = np.max(FSiList)
    FSiMin = np.min(FSiList)

    return FSiMax, FSiMin, FSiList

#get Delta (eq 4)
def getDelta(niche_size, epsilon, xj, niche):
    total = np.zeros_like(xj)
    for i in range(niche_size):
        xi = niche[i]
        total += np.abs(xi-xj)/(niche_size - 1)
    return total*epsilon


In [3]:
######### ALGORITHMS 

In [4]:
#Algo 1 Clustering for Crowding
def crowd(f, population, f_population_size, niche_size, no_of_niches, last_niche_size):
   #Step 1 Generate Reference Point
   ref_pt = np.empty(f.get_dimension())
   for i in range(0, f.get_dimension()):
      ref_pt[i] = np.random.uniform(f.get_lbound(i), f.get_ubound(i)) 
   #Step 1 Calculate distance to Reference Points
   distances = np.empty(( f_population_size))
   for j in range(0, f_population_size):
     distances[j] = Euclidian_distance(ref_pt, population[j], f.get_dimension())
   
   sorted_indices = np.argsort(distances) # get the sorted indices of distances list

   pop = population.copy()

   niches = np.empty((0, niche_size, f.get_dimension()))
   #Step 2
   #Here we are making niches by grouping them based on how close they are to the reference point
   index = 0

   for i in range(0, no_of_niches - 1):
      niche = np.empty((niche_size, f.get_dimension()))
      for j in range(0, niche_size):
         niche[j] = pop[sorted_indices[index]]
         index += 1
      niches = np.append(niches, [niche], axis = 0)

   # ignoring the last niche because it has a mismatched size
   # niche = np.empty((last_niche_size, f.get_dimension()))
   # for j in range(0, last_niche_size):
   #    niche[j] = pop[sorted_indices[index]]
   #    index += 1
   # niches = np.append(niches, [niche], axis = 0)

   return niches

   


In [5]:
#Algo 2 Clustering for Crowding based on Fitness Evaluation
def crowd_fitness(f, population, f_population_size, niche_size, no_of_niches, last_niche_size, fitness_evals):
    # Step 1: Sort fitness evaluations and get sorted indices
    sorted_indices = np.argsort(fitness_evals)  # Indices of sorted fitness values

    # Step 1.1: Sort the population using the sorted indices
    sorted_population = population[sorted_indices]

    niches = []  # Holds all the niche arrays created later
    index = 0  # Tracks the position in the sorted population

    # Step 2: Create niches for all but the last group
    for i in range(no_of_niches - 1):
        niche = sorted_population[index:index + niche_size]  # Slice the sorted population
        niches.append(niche)  # Add the niche to the list
        index += niche_size

    # Step 3: Handle the last niche with the remaining population
    # last_niche = sorted_population[index:index + last_niche_size]
    # niches.append(last_niche)

    return np.array(niches)


In [6]:
#Algo 4 Solution Construction for Ants
def SolConAnts(niches_set, niche_size, FSmax, FSmin, f, epsilon, eta):

    F_const = np.random.rand()

    #Step 1
    for niche in niches_set:
        #Step 1.1
        FSimax, FSimin, FSiList = FSmaxMinOfNiche(niche, f)
        #Step 1.2 
        
        sigma = 0.1 + 0.3 * np.exp(- (FSimax - FSimin) / (FSmax - FSmin + eta))
        #Step 1.3
        #rank of the solution sorted in descending order according to fitness values
        ranks = np.argsort(np.argsort(-np.array(FSiList))) + 1

        #equation 2
        weights = np.array([
            (1 / (sigma * np.sqrt(2 * np.pi) * niche_size)) * np.exp(-((rank - 1) ** 2) / (2 * (sigma ** 2) * niche_size ** 2))
            for rank in ranks
        ])

        #equation 1
        probabilities = weights / np.sum(weights)

        #Step 1.4
        sols = []
        for i in range(0, niche_size):
            #1.4.1
            #xj found using roulette wheel selection
            cumulative_probs = np.cumsum(probabilities)
            j = np.searchsorted(cumulative_probs, np.random.rand())

            xj = niche[j]
            #1.4.2
            if(np.random.rand() < 0.5):
                mu = xj
            else:
                #Get best seed based on which has the best fitness Value
                x_seed = niche[np.argmax(FSiList)]
                mu = xj + F_const * (x_seed - xj)
            #1.4.3
            delta = getDelta(niche_size, epsilon, xj, niche)
            #1.4.4
            sol = np.random.normal(loc=mu, scale=delta) #instead of building our own gaussian/normal dist ftn
                                                        #we use numpy's as it's more efficient
            sols.append(sol)
    return np.array(sols)

            
            

In [7]:
#Algo 5
def local_search(seeds, fitness_values, std_dev, num_sampled_ind, eta, f):
    num_sampled_ind = int(num_sampled_ind)
    #step 1
    FSEmax = max(fitness_values)
    FSEmin = min(fitness_values)
    flag = False

    #step 2
    if(FSEmin<=0):
        FSEmax = FSEmax + np.abs(FSEmin) + eta
        flag = True
    
    #step 3
    probSet = np.array([])
    for i, seed in enumerate(seeds):
        if(flag):
            prob = (fitness_values[i] + np.abs(FSEmin) + eta)/(FSEmax+np.abs(FSEmin) + eta)
        else:
            prob = fitness_values[i] / FSEmax
        probSet = np.append(probSet, prob)

    
    #step 4
    for i, seed in enumerate(seeds):
        if(np.random.rand()<=probSet[i]):
            for _ in range(num_sampled_ind):
                LSj = seed + np.random.normal(0, std_dev, size=seed.shape)
                if(f.evaluate(LSj)>fitness_values[i]):
                    seeds[i] = LSj
                    fitness_values[i] = f.evaluate(LSj)
    return seeds, fitness_values


In [8]:
#Algo 6
def LAMCACO(f, f_population_size, neighbor_size_set, epsilon, eta, local_std_value, num_sampled_ind):
    pop_fit_eval_results = np.empty(f_population_size)
    NoOfFES = 0
    ########### Step 1
    # create and init population with random values in range of lower and upper bounds of ftn
    population = np.zeros((f_population_size, f.get_dimension()))
    for j in range(0, f.get_dimension()):
        population[:, j] = np.random.uniform(f.get_lbound(j), f.get_ubound(j), size=f_population_size) 

    # Compute fitness using function
    for i in range(0, f_population_size):
        pop_fit_eval_results[i] = f.evaluate(population[i])
    NoOfFES += 1
    ########### Step 1 END
    
    ########### Step 2 
    FSmax = np.max(pop_fit_eval_results)
    FSmin = np.min(pop_fit_eval_results)
    ########### Step 2 End
  
    ########### Step 3
    N_nearest = np.random.choice(neighbor_size_set) #randomly select a niche size from the niche size pool  
    ########### Step 3 END

    ########### Step 4 
    niches_num = int(f_population_size / N_nearest) #compute the number of niches

    if f_population_size % N_nearest == 0:
        last_niche_size = 0
    
    else: #if the remainder is not 0, then put all the remained individuals as a new niche
        niches_num += 1
        last_niche_size = f_population_size % N_nearest 

    niches_set = crowd(f, population, f_population_size, N_nearest, niches_num, last_niche_size)  
    ########### Step 4 END

    ########### Step 5
    sols_set = SolConAnts(niches_set, N_nearest, FSmax, FSmin, f, epsilon, eta)
    NoOfFES += N_nearest
    ########### Step 5 END

    ########### Step 6
    #calculating fitness of solutions
    sol_fitness_list = np.array([])
    for sol in sols_set:
        if sol.size==1:
            sol_fitness_list = np.append(sol_fitness_list, f.evaluate(np.array([sol])))
        else:
            sol_fitness_list = np.append(sol_fitness_list, f.evaluate(np.array(sol)))
    NoOfFES += 1
    # Checking which solution is better
    for i, sol in enumerate(sols_set):
        # Find the nearest solution in the population
        nearest_sol_index = np.argmin(np.linalg.norm(population - sol, axis=1))

        # Compare the fitness of the current solution with the nearest solution in the population
        if(sol_fitness_list[i] is None):
            sol_fitness_list[i] = 0

        if sol_fitness_list[i] > pop_fit_eval_results[nearest_sol_index]:
            # Update the population with the better solution
            population[nearest_sol_index] = sol

            # Update the population fitness list
            pop_fit_eval_results[nearest_sol_index] = sol_fitness_list[i]
    ########### Step 6 END

    ########### Step 7
    seeds, fitness_values = local_search(population, pop_fit_eval_results, num_sampled_ind ,local_std_value, eta, f)
    ########### Step 7 END

    return seeds, fitness_values, NoOfFES
    ###########Debug`
    # print("--------------")
    # print(population.shape)
    # print(niches_set.shape)
    # print(sols_set.shape)
    # print(sol_fitness_list.shape)
    # print("--------------")
    ###########Debug END
    




In [9]:
#algo 7 Local Search-Based AMS-ACO (LAMS-ACO) based on fitness ONLY
def LAMSACO(f, f_population_size, neighbor_size_set, epsilon, eta, local_std_value, num_sampled_ind):
    pop_fit_eval_results = np.empty(f_population_size)
    NoOfFES = 0
    
    ########### Step 1
    # Create and initialize the population with random values within function bounds
    population = np.zeros((f_population_size, f.get_dimension()))
    for j in range(f.get_dimension()):
        population[:, j] = np.random.uniform(f.get_lbound(j), f.get_ubound(j), size=f_population_size)

    # Compute fitness of the population
    for i in range(f_population_size):
        pop_fit_eval_results[i] = f.evaluate(population[i])
    NoOfFES += f_population_size
    ########### Step 1 END

    ########### Step 2 
    FSmax = np.max(pop_fit_eval_results)
    FSmin = np.min(pop_fit_eval_results)
    ########### Step 2 END

    ########### Step 3
    # Randomly select a niche size from the given set
    N_nearest = np.random.choice(neighbor_size_set)
    ########### Step 3 END

    ########### Step 4
    # Compute the number of niches
    niches_num = int(f_population_size / N_nearest)
    last_niche_size = f_population_size % N_nearest

    # Adjust the number of niches if the last niche is non-zero
    if last_niche_size != 0:
        niches_num += 1

    # Call the fitness-based niching function
    niches_set = crowd_fitness(f, population, f_population_size, N_nearest, niches_num, last_niche_size, pop_fit_eval_results)
    ########### Step 4 END

    ########### Step 5
    # Generate solutions using ant-based solution construction
    sols_set = SolConAnts(niches_set, N_nearest, FSmax, FSmin, f, epsilon, eta)
    NoOfFES += N_nearest
    ########### Step 5 END

    ########### Step 6
    # Evaluate fitness of solutions
    sol_fitness_list = np.array([])
    for sol in sols_set:
        if sol.size == 1:
            sol_fitness_list = np.append(sol_fitness_list, f.evaluate(np.array([sol])))
        else:
            sol_fitness_list = np.append(sol_fitness_list, f.evaluate(np.array(sol)))
    NoOfFES += len(sols_set)

    # Compare and update the population with better solutions
    for i, sol in enumerate(sols_set):
        nearest_sol_index = np.argmin(np.linalg.norm(population - sol, axis=1))
        if sol_fitness_list[i] > pop_fit_eval_results[nearest_sol_index]:
            population[nearest_sol_index] = sol
            pop_fit_eval_results[nearest_sol_index] = sol_fitness_list[i]
    ########### Step 6 END

    ########### Step 7
    # Perform local search to refine solutions
    seeds, fitness_values = local_search(population, pop_fit_eval_results, num_sampled_ind, local_std_value, eta, f)
    ########### Step 7 END

    return seeds, fitness_values, NoOfFES


In [10]:
#setting up vars
population_size_set = np.array([80,80,80,80,80,100,300,300,300,100]) #the population size setting for each function
neighbor_size_set = np.array([2,4,8,12,16,20])#the niche size set
                                              #how many crowds should be created for each pop
                                              #Denoted as input G in algo 6 and 7
funToRun = 10
epsilon_set = [1e-4]#[1e-1,1e-2,1e-3,1e-4,1e-5]
local_std_value = 1e-4
eta = 1e-10
num_sampled_ind = 2
noOfRuns = 1

In [11]:
#Main for LAMCACO
for fun in range(1, funToRun + 1):
    f = CEC2013(fun) # building function
    f_population_size = population_size_set[fun - 1]
    
    for epsilon in epsilon_set:
        print(f"Running function: f{fun} with epsilon: {epsilon}" )
        sumOfGlobalOptima = 0
        sumOfSucceses = 0
        highestGlobalOptima = 0
        TotalFESCarriedOut = 0
        for run in range(noOfRuns):
            NoOfFesi = 0
            MaxFesAllowed = f.get_maxfes()
            globalOptimaFoundNotTuple = 0

            while(NoOfFesi<MaxFesAllowed):
                population, fitness_values, NoOfFESL = LAMCACO(f, f_population_size, neighbor_size_set, epsilon, eta, local_std_value, num_sampled_ind)
                NoOfFesi += NoOfFESL
                
                #Store highest number of global optima found in case later iteration has less luck

                globalOptimaFound = how_many_goptima(population, f, epsilon)
                globalOptimaFoundNotTuple = globalOptimaFound[0]
                if(globalOptimaFoundNotTuple > highestGlobalOptima):
                    highestGlobalOptima = globalOptimaFoundNotTuple

                #if all the global optima has been found, break
                if globalOptimaFound == f.get_no_goptima():
                    break  
            
            if(highestGlobalOptima == f.get_no_goptima()):
                sumOfSucceses += 1
            
            sumOfGlobalOptima += highestGlobalOptima
            TotalFESCarriedOut += NoOfFesi  

        PR = sumOfGlobalOptima/(f.get_fitness_goptima()*noOfRuns)
        SR = sumOfSucceses/noOfRuns
        CS = NoOfFesi/noOfRuns
        print(f"PR: {PR}    SR: {SR}   CS: {CS}")    

        


Running function: f1 with epsilon: 0.0001


/var/folders/yn/sz7b_m1n2wg127k9bqyqdxsw0000gn/T/ipykernel_63517/3271661248.py:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pop_fit_eval_results[i] = f.evaluate(population[i])


PR: 0.0    SR: 0.0   CS: 50008.0
Running function: f2 with epsilon: 0.0001
PR: 3.0    SR: 0.0   CS: 50002.0
Running function: f3 with epsilon: 0.0001
PR: 1.0    SR: 1.0   CS: 50014.0
Running function: f4 with epsilon: 0.0001
PR: 0.0    SR: 0.0   CS: 50004.0
Running function: f5 with epsilon: 0.0001
PR: 0.9693412358074444    SR: 0.0   CS: 50012.0
Running function: f6 with epsilon: 0.0001
PR: 0.0    SR: 0.0   CS: 200004.0
Running function: f7 with epsilon: 0.0001
PR: 2.0    SR: 0.0   CS: 200002.0
Running function: f8 with epsilon: 0.0001
PR: 0.0    SR: 0.0   CS: 400000.0
Running function: f9 with epsilon: 0.0001
PR: 1.0    SR: 0.0   CS: 400006.0
Running function: f10 with epsilon: 0.0001
PR: -0.5    SR: 0.0   CS: 200000.0


In [12]:
#Main for LAMSACO
for fun in range(1, funToRun + 1):
    f = CEC2013(fun) # building function
    f_population_size = population_size_set[fun - 1]
    
    for epsilon in epsilon_set:
        print(f"Running function: f{fun} with epsilon: {epsilon}" )
        sumOfGlobalOptima = 0
        sumOfSucceses = 0
        highestGlobalOptima = 0
        TotalFESCarriedOut = 0
        for run in range(noOfRuns):
            NoOfFesi = 0
            MaxFesAllowed = f.get_maxfes()
            globalOptimaFoundNotTuple = 0

            while(NoOfFesi<MaxFesAllowed):
                population, fitness_values, NoOfFESL = LAMSACO(f, f_population_size, neighbor_size_set, epsilon, eta, local_std_value, num_sampled_ind)
                NoOfFesi += NoOfFESL
                
                #Store highest number of global optima found in case later iteration has less luck

                globalOptimaFound = how_many_goptima(population, f, epsilon)
                globalOptimaFoundNotTuple = globalOptimaFound[0]
                if(globalOptimaFoundNotTuple > highestGlobalOptima):
                    highestGlobalOptima = globalOptimaFoundNotTuple

                #if all the global optima has been found, break
                if globalOptimaFound == f.get_no_goptima():
                    break  
            
            if(highestGlobalOptima == f.get_no_goptima()):
                sumOfSucceses += 1
            
            sumOfGlobalOptima += highestGlobalOptima
            TotalFESCarriedOut += NoOfFesi  

        PR = sumOfGlobalOptima/(f.get_fitness_goptima()*noOfRuns)
        SR = sumOfSucceses/noOfRuns
        CS = NoOfFesi/noOfRuns
        print(f"PR: {PR}    SR: {SR}   CS: {CS}")    

        


Running function: f1 with epsilon: 0.0001


/var/folders/yn/sz7b_m1n2wg127k9bqyqdxsw0000gn/T/ipykernel_63517/2830909127.py:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pop_fit_eval_results[i] = f.evaluate(population[i])


PR: 0.0    SR: 0.0   CS: 50032.0
Running function: f2 with epsilon: 0.0001
PR: 3.0    SR: 0.0   CS: 50100.0
Running function: f3 with epsilon: 0.0001
PR: 1.0    SR: 1.0   CS: 50012.0
Running function: f4 with epsilon: 0.0001
PR: 0.0    SR: 0.0   CS: 50008.0
Running function: f5 with epsilon: 0.0001
PR: 0.9693412358074444    SR: 0.0   CS: 50072.0
Running function: f6 with epsilon: 0.0001
PR: 0.0    SR: 0.0   CS: 200108.0
Running function: f7 with epsilon: 0.0001
PR: 1.0    SR: 0.0   CS: 200288.0
Running function: f8 with epsilon: 0.0001
PR: 0.0    SR: 0.0   CS: 400240.0
Running function: f9 with epsilon: 0.0001
PR: 0.0    SR: 0.0   CS: 400264.0
Running function: f10 with epsilon: 0.0001
PR: -0.0    SR: 0.0   CS: 200068.0
